# Monitoring model explanations
In questo notebook testiamo la funzionalita' model explainability monitoring di SageMaker 

In [ ]:
import copy
import json
import random
import time
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.clarify import (
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.model import Model
from sagemaker.model_monitor import (
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelExplainabilityMonitor,
)
from sagemaker.s3 import S3Downloader, S3Uploader
role = 'arn:aws:iam::088093517335:role/service-role/AmazonSageMaker-ExecutionRole-20210425T181732'

Inizializziamo una nuova sessione sagemaker e i specifichiamo in quali buckets andra' a lavorare il monitor

In [2]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

bucket =  Session().default_bucket()
print(f"Demo Bucket: {bucket}")
prefix = 'sagemaker/DEMO-ClarifyModelMonitor-20200901'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

s3_capture_upload_path = f'{s3_key}/datacapture'
#ground_truth_upload_path = f"{s3_key}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
#print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

baseline_results_uri = f'{s3_key}/baselining'
print(f'Baseline results uri: {baseline_results_uri}')

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.large"


AWS region: eu-central-1
Demo Bucket: sagemaker-eu-central-1-088093517335
S3 key: s3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901
Capture path: s3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901/datacapture
Report path: s3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901/reports
Baseline results uri: s3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901/baselining


In [3]:
model_file = "model/xgb-churn-prediction-model.tar.gz"
test_file = 'test_data/upload-test-file.txt'
test_dataset = "test_data/test-dataset-input-cols.csv"
validation_dataset = "test_data/validation-dataset-with-header.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

Usiamo lo stesso endpoint di prima, il nome e' trovabile anche da dashboard SageMaker

In [20]:
endpoint_name = 'DEMO-xgb-churn-pred-model-monitor'
#arn:aws:sagemaker:eu-central-1:088093517335:model/

Inizializziamo il monitor

In [6]:
model_explainability_monitor = ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

In [7]:


model_explainability_baselining_job_result_uri = f"{baseline_results_uri}/model_explainability"
model_explainability_data_config = DataConfig(
    s3_data_input_path=validation_dataset,
    s3_output_path=model_explainability_baselining_job_result_uri,
    label=label_header,
    headers=all_headers,
    dataset_type=dataset_type,
)



Usando SHAP bisogna specificare una baseline (valori da usare quando feature e' considerata assente). Dobbiamo inoltre creare un endpoint parallelo per la generazione di spiegazioni usando ModelConfig.

E' importante usare lo stesso modello su cui si basa l'endpoint da monitorare, il nome e' trovabile nella dashboard di SageMaker

In [8]:
test_dataframe = pd.read_csv(test_dataset, header=None)
shap_baseline = [list(test_dataframe.mean())]
model_name = f"sagemaker-xgboost-2021-05-07-11-50-04-331"


model_config = ModelConfig(
    model_name=model_name,
    instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    content_type=dataset_type,
    accept_type=dataset_type,
)

shap_config = SHAPConfig(
    baseline=shap_baseline,
    num_samples=100,
    agg_method="mean_abs",
    save_local_shap_values=False,
)

In [9]:
model_explainability_monitor.suggest_baseline(
    data_config=model_explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)
print(f"ModelExplainabilityMonitor baselining job: {model_explainability_monitor.latest_baselining_job_name}")


Job Name:  baseline-suggestion-job-2021-05-10-09-55-30-578
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-central-1-088093517335/baseline-suggestion-job-2021-05-10-09-55-30-578/input/dataset/validation-dataset-with-header.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901/baselining/model_explainability/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonit

In [10]:
model_explainability_monitor.latest_baselining_job.wait(logs=False)
model_explainability_constraints = model_explainability_monitor.suggested_constraints()
print()
print(f"ModelExplainabilityMonitor suggested constraints: {model_explainability_constraints.file_s3_uri}")
print(S3Downloader.read_file(model_explainability_constraints.file_s3_uri))


..........................................................................................................................................!
ModelExplainabilityMonitor suggested constraints: s3://sagemaker-eu-central-1-088093517335/sagemaker/DEMO-ClarifyModelMonitor-20200901/baselining/model_explainability/analysis.json
{
    "version": "1.0",
    "explanations": {
        "kernel_shap": {
            "label0": {
                "global_shap_values": {
                    "Account Length": 0.006796025841907208,
                    "VMail Message": 0.01597435086401014,
                    "Day Mins": 0.038564491528327534,
                    "Day Calls": 0.00706242235138291,
                    "Eve Mins": 0.014755214861905543,
                    "Eve Calls": 0.006438401036630888,
                    "Night Mins": 0.009508115182763595,
                    "Night Calls": 0.010242051675769418,
                    "Intl Mins": 0.010200413902228212,
                    "Intl Calls": 0.01028

Creato il monitor possiamo definire una schedule cosi' come abbiamo fatto per l'esercizio di prima

In [15]:
schedule_expression = CronExpressionGenerator.hourly()

model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=s3_report_path,
    endpoint_input=endpoint_name,
    schedule_cron_expression=schedule_expression,
)

Mandiamo di nuovo un po' di richieste all'endpoint e aspettiamo che una pipeline di monitoraggio venga completata

In [17]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import time

predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())


with open('test_data/test_sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        print(payload)
        response = predictor.predict(data=payload)
        time.sleep(1)
        
print("Done!")

186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55
132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
112,17,183.2,95,252.8,125,156.7,95,9.7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
91,24,93.5,112,183.4,128,240.7,133,9.9,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
22,0,110.3,107,166.5,93,202.3,96,9.5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
102,0,186.8,92,173.7,123,250.9,131,9.7,4,2,0

125,0,191.6,115,205.6,108,210.2,123,9.2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
128,0,245.2,112,101.5,101,152.3,116,10.7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
65,0,153.9,117,220.1,122,280.5,147,8.5,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
209,0,255.1,124,230.6,110,218.0,69,8.5,5,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
90,0,179.1,71,190.6,81,127.7,91,10.6,7,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
105,0,125.4,116,261.5,95,241.6,104,11.4,9,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
61,0,260.0,123,210.5,127,234.7,70,9.0,3,1,0,

171,0,137.5,110,198.1,109,292.7,131,13.3,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
104,0,130.5,77,131.2,117,264.7,63,13.0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
50,0,295.3,127,127.4,100,166.8,105,9.6,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
99,0,254.4,120,159.3,92,264.4,94,6.0,5,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
16,0,209.5,89,172.8,85,94.1,102,8.8,4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
11,24,131.5,98,230.2,111,283.7,87,10.0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
120,0,185.7,133,235.1,149,256.4,78,16.9,6,0,0,0,0

120,0,178.4,97,168.3,113,120.5,93,9.3,9,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
91,0,153.0,123,141.1,127,171.5,76,10.3,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
90,0,175.9,111,285.2,115,150.8,122,13.0,7,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
93,0,328.1,106,151.7,89,303.5,114,8.7,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
85,0,235.8,109,157.2,94,188.2,99,12.0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
48,0,171.9,98,159.0,127,139.5,101,7.6,3,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
93,42,152.3,90,267.5,102,266.9,130,11.3,5,7,0,0,